In [2]:
!pip3 install sklearn

    100% |################################| 20.0MB 74kB/s  eta 0:00:01
    100% |################################| 25.9MB 52kB/s  eta 0:00:01
    100% |################################| 307kB 4.2MB/s eta 0:00:01
  Running setup.py bdist_wheel for sklearn ... done
  Stored in directory: /root/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [65]:
import sys
import numpy as np
import pandas as pd
from scipy.misc.pilutil import imread
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

#import cv2
import time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng

In [409]:
# a=np.array([106,21,5,219,2,82])
n = 100
XTrain=np.zeros(shape=(n,8))
YTrain=np.zeros(shape=(n))
for i in range(0, n):
    XTrain[i] = list(bin(int(i))[2:].zfill(8))
    YTrain[i] = i%3

In [410]:
XTrain

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 1.],
       [0., 0., 0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 1., 1.],
       [0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 1.],
       [0., 0., 0., 0., 1., 1., 1., 0.],
       [0., 0., 0., 0., 1., 1., 1., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 1., 1.],
       [0., 0., 0., 1., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 1., 0., 1.],
       [0., 0., 0., 1., 0., 1., 1., 0.],
       [0., 0., 0., 1., 0., 1., 1., 1.],
       [0., 0., 

In [551]:
XTrain.shape

(100, 8)

In [411]:
YTrain

array([0., 1., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2., 0., 1.,
       2., 0., 1., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2., 0.,
       1., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2.,
       0., 1., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2., 0., 1.,
       2., 0., 1., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2., 0.,
       1., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2., 0.])

In [412]:
def generateBatch(xTrain, yTrain, batchSize):
    inputLLs = []
    inputRLs = []
    targetLs = []
    gr0Index = 0
    gr1Index = 1
    classes = np.unique(yTrain)
#     print("classes:",classes)
    chooseDiffClasses = [0,0]
    chooseDiffClasses[0] = rng.choice(classes)
    chooseDiffClasses[1] = rng.choice([i for i in classes if i not in [chooseDiffClasses[0]]])
    
    chooseSameClasses = rng.choice(classes)
    
    numPerChoose = batchSize // 2

#     print("chooseDiffClasses:",chooseDiffClasses)
#     print("chooseSameClasses:",chooseSameClasses)
    
    #For same class
#     print("For same class")
    
    indexSCLs = np.where(yTrain==chooseSameClasses)[0]
    rng.shuffle(indexSCLs)
#     print("indexSCLs:",indexSCLs)
    
    indexSC1 = rng.choice(indexSCLs[0:len(indexSCLs)//2],numPerChoose)
    indexSC2 = rng.choice(indexSCLs[len(indexSCLs)//2:len(indexSCLs)],numPerChoose)
#     print("indexSC1:",indexSC1)
#     print("indexSC2:",indexSC2)
    inputLLs = xTrain[indexSC1]
    inputRLs = xTrain[indexSC2]
    targetLs = np.full((1, numPerChoose), 1)[0]
    
#     print(inputLLs)
#     print(inputRLs)
#     print(targetLs)
    
    #For diff classes
#     print("For diff class")

    indexSC1Ls = np.where(yTrain==chooseDiffClasses[0])[0]
    rng.shuffle(indexSC1Ls)
    indexSC2Ls = np.where(yTrain==chooseDiffClasses[1])[0]
    rng.shuffle(indexSC2Ls)
    
    indexSC1 = rng.choice(indexSC1Ls,numPerChoose)
    indexSC2 = rng.choice(indexSC2Ls,numPerChoose)
    
    inputLLs = np.append(inputLLs, xTrain[indexSC1], axis=0)
    inputRLs = np.append(inputRLs, xTrain[indexSC2], axis=0)
    targetLs = np.append(targetLs,np.full((1, numPerChoose), 0)[0])
#     print(inputLLs)
#     print(inputRLs)
#     print(targetLs)
    
#     for i in choosedClasses:
        
#     for i in range(0, len(yTrain)):
#         for j in range(i+1, len(yTrain)):
#             ck = int(yTrain[i]==yTrain[j])
# #             print(xTrain[i].reshape(8,1))
#             inputLLs.append(xTrain[i].reshape(8,1))
#             inputRLs.append(xTrain[j].reshape(8,1))
# #             print(yTrain[i]," ",yTrain[j]," ",ck)
#             targetLs.append(ck)
    inputLLs = inputLLs.reshape(inputLLs.shape[0],8,1)
    inputRLs = inputRLs.reshape(inputRLs.shape[0],8,1)

    return ((inputLLs, inputRLs), targetLs)
# 
(inputs,targets) = generateBatch(XTrain,YTrain,4)

In [361]:
print(inputs[0].shape)

(4, 8, 1)

In [439]:
# We have Weight matrix, W, d x z
model = linear_model.LogisticRegression(random_state=1)
model.fit(X_train, y_train)
W = model.coef_.T

In [534]:
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Dense(6, activation='sigmoid'))
    model.add(Flatten())
    model.add(Dense(3, activation='sigmoid'))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid')(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [535]:
model = get_siamese_model((8, 1))
model.summary()

Model: "functional_37"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_39 (InputLayer)           [(None, 8, 1)]       0                                            
__________________________________________________________________________________________________
input_40 (InputLayer)           [(None, 8, 1)]       0                                            
__________________________________________________________________________________________________
sequential_19 (Sequential)      (None, 3)            159         input_39[0][0]                   
                                                                 input_40[0][0]                   
__________________________________________________________________________________________________
lambda_18 (Lambda)              (None, 3)            0           sequential_19[0][0]  

In [536]:
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

In [537]:
def getRandom(arr,excepts=[]):
    return rng.choice([i for i in arr if i not in excepts])

In [547]:
print(inputs[0].shape)
print(inputs[1].shape)
print(targets)

(20, 8, 1)
(20, 8, 1)
[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]


In [548]:
epoch=1000
batchSize=20
for i in range(0,epoch):
    (inputs,targets) = generateBatch(XTrain,YTrain,batchSize)
    loss = model.train_on_batch(inputs, targets)
    print(loss)

0.697365403175354
0.6914330124855042
0.6922457218170166
0.6959626078605652
0.6936318278312683
0.6934208869934082
0.6929982304573059
0.6979981660842896
0.6943879127502441
0.6940805315971375
0.6925228834152222
0.6925780177116394
0.6891199946403503
0.6936677694320679
0.6934161186218262
0.692773699760437
0.6949936747550964
0.6962010264396667
0.6943359971046448
0.6949589848518372
0.6922491192817688
0.6933517456054688
0.6904087066650391
0.6958338618278503
0.6958473324775696
0.6942882537841797
0.6935323476791382
0.6921451687812805
0.691379725933075
0.6888753771781921
0.6948295831680298
0.6919615864753723
0.6958943605422974
0.6927794218063354
0.6974552869796753
0.6926974058151245
0.6894570589065552
0.6923429369926453
0.6956990957260132
0.6955528259277344
0.6968340873718262
0.6930042505264282
0.6949253082275391
0.6915671825408936
0.6932748556137085
0.6897072792053223
0.6961031556129456
0.6926445364952087
0.6944260597229004
0.6927586793899536
0.6964435577392578
0.693557858467102
0.69367158412933

0.6943153142929077
0.6954353451728821
0.6866881847381592
0.6915132403373718
0.692578911781311
0.6900246739387512
0.6933035850524902
0.6942970752716064
0.6950330138206482
0.6920219659805298
0.6971955299377441
0.6929251551628113
0.69373619556427
0.6941820383071899
0.6953469514846802
0.691438615322113
0.6953074336051941
0.691170871257782
0.6917895674705505
0.6947712898254395
0.6914323568344116
0.6918746829032898
0.6931963562965393
0.692438006401062
0.6887749433517456
0.694183349609375
0.6907772421836853
0.6898356676101685
0.696733832359314
0.6959144473075867
0.6941660642623901
0.6906856894493103
0.6922159790992737
0.6914269328117371
0.6921359300613403
0.694022536277771
0.6915342211723328
0.6933115720748901
0.6945303082466125
0.6925092935562134
0.6921585202217102
0.6943877935409546
0.6943848729133606
0.6961197853088379
0.6938718557357788
0.6915432810783386
0.6932536959648132
0.6940523386001587
0.6932834982872009
0.6931926608085632
0.6914759874343872
0.6922845244407654
0.6917956471443176
0.

0.6929577589035034
0.6894792318344116
0.6899289488792419
0.6946648359298706
0.6936528086662292
0.6909832954406738
0.6945748925209045
0.6938600540161133
0.6928938031196594
0.6936629414558411
0.6923394203186035
0.6928077936172485
0.6915956735610962
0.6880370378494263
0.6945733428001404
0.691504180431366
0.6914125680923462
0.6909200549125671
0.6892444491386414
0.6894286870956421
0.694603443145752
0.6919956803321838
0.6969431638717651
0.6939336657524109
0.6918123960494995
0.6923593878746033
0.6940490007400513
0.6944241523742676
0.6904205083847046
0.6891918182373047
0.6927824020385742
0.69124436378479
0.6942868828773499
0.697527289390564
0.6892653703689575
0.6953611373901367
0.6943619847297668
0.6904875040054321
0.6911302804946899
0.699741780757904
0.697121798992157
0.6910306215286255
0.695184588432312
0.6932190656661987
0.6939131021499634
0.6944392323493958
0.6952594518661499
0.6919782757759094
0.6933215260505676
0.6953575015068054
0.690910816192627
0.6920149922370911
0.6939001083374023
0.

In [549]:
np.argmax([1,2,5])

2

In [550]:
def compareArr(A,B):
    number_of_equal_elements = np.sum(A==B)
#     print(number_of_equal_elements)
    total_elements = len(A)
    percentage = number_of_equal_elements/total_elements
    return (number_of_equal_elements,total_elements,percentage)

def testOneShot():
    accCount = 0
    total = 0
    testNum = 1
    for i in range(0,testNum):
        inputs, targets = generateBatch(XTrain,YTrain,batchSize)
        probs = model.predict(inputs)
#         print(probs.shape)
        probs = probs.reshape(1,len(probs))[0]
#         print(probs)
        for j in range(0,len(probs)):
            if (probs[j] < 0.5):
                probs[j] = 0
            else:
                probs[j] = 1
#         print(targets)
#         print(probs)
        (c,t,p) = compareArr(probs,targets)
#         print(c,t,p)
        accCount = accCount + c
        total = total + t
    print(accCount,total)
    print("Percent:", (accCount*100)/total)

In [544]:
testOneShot()

10 20
Percent: 50.0
